In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.multioutput import ClassifierChain
from sklearn.svm import SVC
from gensim.models.doc2vec import Doc2Vec
import re
import jieba
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib 
from sklearn.metrics import f1_score
import random

divorce_model=Doc2Vec.load("divorce/divorce.model") #divorce的Doc2Vec模型
loan_model=Doc2Vec.load("loan/loan.model")#loan的Doc2Vec模型
labor_model=Doc2Vec.load("labor/labor.model")#labor的Doc2Vec模型

data_input = {1: "divorce/data.txt", 2: "labor/data.txt", 3: "loan/data.txt"}
models={1:divorce_model,2:loan_model,3:labor_model}
data_type = {1: "divorce", 2: "labor", 3: "loan"}
label_size = 20
test_ratio = 0.3
punction = "！？。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."

def line_processing(line):  # 提取每行数据的文本内容
    line = line.strip().split('\t')
    sentence = line[1]
    sentence = re.sub(r'[{}]'.format(punction),' ',sentence).split(' ')
    sent=[]
    for sub_sentence in sentence:
        if sub_sentence!='':
            sent.extend(list(jieba.cut(sub_sentence)))
    return line[0], sent, line[2]

def constructDataSet(data_path,model_tag): #构建X，Y的数据集
    data_file = open(data_path,'r',encoding='utf-8')
    lines = data_file.read().splitlines()
    X=[]
    Y=[]
    d2v = models[model_tag]
    for line in lines:
        _,x,y = line_processing(line)
        x=d2v.infer_vector(x)
        X.append(x)
        y = list(map(int,y.split()))
        Y.append(y)
    Y = np.array(Y)
    data_file.close()
    return X,Y

def splitDataSet(X,Y,modify=False,ratio=.2): #构建训练集和测试集
    if modify: #调整正负样本比例
        X_true=[]
        X_false=[]
        Y_true=[]
        Y_false=[]
        for i in range(len(Y)):
            if Y[i]==1:
                X_true.append(X[i])
                Y_true.append(Y[i])
            else:
                X_false.append(X[i])
                Y_false.append(Y[i])
        true_num = len(X_true)
        false_num = true_num*ration
        for i in range(0,len(X_false),int(len(X_false)/false_num)):
            X_true.append(X_false[i])
            Y_true.append(Y_false[i])
        X_train,X_test,Y_train,Y_test =  train_test_split(X_true,Y_true,test_size=test_ratio,shuffle=True)
    else:
        X_train,X_test,Y_train,Y_test =  train_test_split(X,Y,test_size=test_ratio,shuffle=True)
    return X_train,X_test,Y_train,Y_test



/home/rpj/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def beginTrain():
    for i in range(1,4):
        print(data_input[i].split("/")[0])
        X,Y = constructDataSet(data_input[i],i)
        X_train,X_test,Y_train,Y_test = splitDataSet(X,Y)
        base_classifier = SVC(gamma='auto')
        chains = [ClassifierChain(base_classifier,order='random',random_state=j) for j in range(10)]
        for chain in chains:
            chain.fit(X_train,Y_train)
        chain_predit = np.array([chain.predict(X_test) for chain in chains])
        predict_label = chain_predict.mean(axis=0)
        res = []
        for j in predict_label:
            if j>.5:
                res.append(1)
            else:
                res.append(0)
        f1score+=f1_score(Y_test,res,average='macro')
        print(f1score)

In [ ]:
X,Y = constructDataSet(data_input[1],1)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.904 seconds.
Prefix dict has been built succesfully.


In [ ]:
X_train,X_test,Y_train,Y_test = splitDataSet(X,Y)